# 01 — Pipeline de datos

Este notebook ejecuta el pipeline completo de datos del TFG "Gold Price Dynamics":

1. **Descarga** de datos de FRED, Yahoo Finance y Google Trends
2. **Limpieza** y transformación de cada serie
3. **Fusión** en el dataset maestro (`gold_macro_monthly.csv`)
4. **Validación** del resultado final

## Prerrequisitos

- FRED API key en `.env` (ver `.env.example`)
- Datos manuales WGC en `data/manual/` (opcional, ver README)
- Dependencias instaladas: `pip install -r requirements.txt`

In [ ]:
import sys
sys.path.insert(0, '..')

import logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

import pandas as pd
import numpy as np

from src.config import *

## 1. Descarga de datos

In [ ]:
from src.data.download import download_all

raw_data = download_all()

## 2. Limpieza y transformación

In [ ]:
from src.data.clean import clean_all

clean_data = clean_all()

print(f"\nSeries procesadas: {len(clean_data)}")
for name, s in clean_data.items():
    print(f"  {name}: {s.notna().sum()} obs válidas, rango [{s.index.min().date()} — {s.index.max().date()}]")

## 3. Fusión en dataset maestro

In [ ]:
from src.data.merge import merge_series, save_master_dataset

df = merge_series(clean_data)
save_master_dataset(df)

print(f"\nDataset maestro: {df.shape[0]} filas × {df.shape[1]} columnas")
df.head()

## 4. Validación

In [ ]:
from src.data.pipeline import validate_dataset

validate_dataset(df)

In [ ]:
# Resumen de NaN por columna
nan_summary = df.isna().sum().to_frame('NaN count')
nan_summary['% NaN'] = (df.isna().sum() / len(df) * 100).round(1)
nan_summary[nan_summary['NaN count'] > 0]

In [ ]:
# Distribución de episodios
df['episode'].value_counts()